# PageRank con MapReduce
## Acosta Imandt Daniel
* Descarga el [grafo de la Web de Stanford](https://snap.stanford.edu/data/web-Stanford.html), programa el algoritmo de PageRank usando MapReduce y calcula las relevancias de las páginas del grafo. Para este programa considera que el vector de relevancia $\mathbf{r}$ cabe en la memoria principal de los nodos que ejecutan las tareas. Compara tus resultados con la función [`pagerank`](https://networkx.org/documentation/stable/reference/algorithms/generated/networkx.algorithms.link_analysis.pagerank_alg.pagerank.html) de la biblioteca [NetworkX](https://networkx.org/documentation/stable/index.html).

* Punto extra: programa el algoritmo de PageRank usando MapReduce, considerando que el vector de relevancia $\mathbf{r}$ no cabe en la memoria principal de los nodos que ejecutan las tareas. 

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
import networkx as nx
from sklearn.preprocessing import Normalizer
import pyspark
from pyspark.sql import SparkSession
from scipy import sparse
spark = SparkSession.builder.appName('pagerank').getOrCreate()

In [3]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()
sparkContext=spark.sparkContext 

In [4]:
data = spark.read.csv('/content/web-Stanford.csv', header = True, inferSchema = True,sep ='\t')
data = data.toPandas()
data

,FromNodeId,ToNodeId
0,1,6548
1,1,15409
2,6548,57031
3,15409,13102
4,2,17794
...,...,...
2312492,281865,186750
2312493,281865,225872
2312494,281888,114388
2312495,281888,192969


In [5]:
#Creamos el grafo
l = list(data.values[:,0])
ll = list(data.values[:,1])
g = nx.DiGraph()
for i,j in enumerate(l):
    g.add_edge(j,ll[i])
nodes = list(g.nodes())
#Creamos la matriz de adyacencia
m_ad = nx.adjacency_matrix(g,nodelist=nodes)
#Sacamos la matriz estocastica a partir de la de adyacencia
norm = Normalizer(norm='l1')
M = norm.fit_transform(m_ad.T).T
#nx.draw(g) muchos nodos para graficar y se tardo :( ,a mi me gusta mucho ver redes pero no pudo funcionar :(

In [6]:
n = M.shape[0]
r = np.ones((n,1)) / n
r

array([[3.54731947e-06],
       [3.54731947e-06],
       [3.54731947e-06],
       ...,
       [3.54731947e-06],
       [3.54731947e-06],
       [3.54731947e-06]])

In [7]:
# Este es el código visto en clase que se utilizara para comparar 
import numpy as np
from scipy.sparse import csr_matrix

def pagerank(M, iter=100):
  n = M.shape[0]
  r = np.ones(n) / n
  for i in range(iter):
    print('Iteración {0}: r = {1}'.format(i,r))
    r = M @ r

  return r

In [8]:
def prmr(M, beta,iter):
    
    # Sacamos a r1 con la 1/n
    n = M.shape[0]
    r1 = np.ones((n,1)) / n
    row, col = M.nonzero()
    a = [(v,col[j],M[v,col[j]]) for j,v in enumerate(row)]
    
    #Paralelizamos
    rdd = sparkContext.parallelize(a)
    
    #Sacamos las iteracciones
    for j in range(iter):
        print('La iteración',j,'es:',r1.T)
        
        #Inicializamos a r_hat 
        r_hat = np.zeros((n,1))
        #Mapeamos
        mul_simple = rdd.map(lambda x: (x[0],x[2]*r1[x[1],0]))
        #Reducimos
        mul_suma = mul_simple.reduceByKey(lambda x,y : x+y)
        #Vamos llenando r_hat
        for j in mul_suma.take(mul_suma.count()):
            r_hat[j[0]] = j[1]
        # multiplicamos r_hat por la beta escogida   
        r_hat = beta*r_hat
        s = (1 - r_hat.sum()) / n
        
        #Llenamos a r
        r1 = r_hat + s
        
    return r1

## Una pequeña prueba con una pequeña matriz


In [9]:
Me = np.array([[0,1/2,1/3,1/4],[1,1/4,1/3,1/4],[0,1/4,0,1/4],[0,0,1/3,1/4]])
G = nx.convert_matrix.from_numpy_array(Me)
print(Me)

[[0.         0.5        0.33333333 0.25      ]
 [1.         0.25       0.33333333 0.25      ]
 [0.         0.25       0.         0.25      ]
 [0.         0.         0.33333333 0.25      ]]


In [10]:
%%timeit
prmr(Me,0.8,20)

La iteración 0 es: [[0.25 0.25 0.25 0.25]]
La iteración 1 es: [[0.26666667 0.41666667 0.15       0.16666667]]
La iteración 2 es: [[0.29       0.42       0.16666667 0.12333333]]
La iteración 3 es: [[0.28711111 0.43511111 0.15866667 0.11911111]]
La iteración 4 es: [[0.29017778 0.43284444 0.16084444 0.11613333]]
La iteración 5 es: [[0.2892563  0.43482963 0.15979556 0.11611852]]
La iteración 6 es: [[0.2897677  0.43420681 0.16018963 0.11583585]]
La iteración 7 es: [[0.28956713 0.43453993 0.16000853 0.1158844 ]]
La iteración 8 es: [[0.2896618  0.43440751 0.16008487 0.11584582]]
La iteración 9 es: [[0.28962147 0.4344694  0.16005067 0.11585846]]
La iteración 10 es: [[0.28963963 0.43444293 0.16006557 0.11585187]]
La iteración 11 es: [[0.2896317  0.43445482 0.16005896 0.11585453]]
La iteración 12 es: [[0.28963522 0.43444962 0.16006187 0.11585329]]
La iteración 13 es: [[0.28963367 0.43445192 0.16006058 0.11585382]]
La iteración 14 es: [[0.28963436 0.43445091 0.16006115 0.11585359]]
La iteración 1

In [11]:
%%timeit
pagerank(Me,20)

Se han truncado las últimas 5000 líneas del flujo de salida.
Iteración 1: r = [0.27083333 0.45833333 0.125      0.14583333]
Iteración 2: r = [0.30729167 0.46354167 0.15104167 0.078125  ]
Iteración 3: r = [0.30164931 0.49305556 0.13541667 0.06987847]
Iteración 4: r = [0.30913628 0.4875217  0.14073351 0.06260851]
Iteración 5: r = [0.30632415 0.49358001 0.13753255 0.0625633 ]
Iteración 6: r = [0.30827501 0.49120416 0.13903583 0.06148501]
Iteración 7: r = [0.30731861 0.49279258 0.13817229 0.06171653]
Iteración 8: r = [0.30788285 0.49200331 0.13862728 0.06148656]
Iteración 9: r = [0.30758239 0.49246441 0.13837247 0.06158073]
Iteración 10: r = [0.30775154 0.49221783 0.13851129 0.06151934]
Iteración 11: r = [0.30765918 0.49235627 0.13843429 0.06155026]
Iteración 12: r = [0.30771046 0.49228057 0.13847663 0.06153233]
Iteración 13: r = [0.30768225 0.49232257 0.13845323 0.06154196]
Iteración 14: r = [0.30769785 0.49229945 0.13846613 0.06153657]
Iteración 15: r = [0.30768925 0.49231223 0.138459   

In [12]:
%%timeit
pr = nx.pagerank(G, alpha=0.80,max_iter=20, tol=1e-06)
pr

1.93 ms ± 423 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


pues hace una buena aproximación para 20 iteracciones:)
El problema es que si toma bastante mas tiempo, aunqie hay que tomar en cuenta que la paraleización y el mapreduce son de mentiritas jajaja, ya que solo cuento con un cluster

## Ahora nos vamos por la buena, Stanford

In [13]:
prmr(M,0.8,10) #Corrio en 3 min

La iteración 0 es: [[3.54731947e-06 3.54731947e-06 3.54731947e-06 ... 3.54731947e-06
  3.54731947e-06 3.54731947e-06]]
La iteración 1 es: [[2.80587425e-06 1.85992239e-06 1.85992239e-06 ... 9.35114240e-07
  9.14819941e-07 1.86318430e-06]]
La iteración 2 es: [[1.80901973e-06 1.56529425e-06 1.56529425e-06 ... 8.58142568e-07
  8.61440716e-07 2.33357008e-06]]
La iteración 3 es: [[1.62975340e-06 1.34460275e-06 1.34460275e-06 ... 8.25830182e-07
  8.27379865e-07 1.69245005e-06]]
La iteración 4 es: [[1.51140438e-06 1.28209151e-06 1.28209151e-06 ... 8.15542851e-07
  8.14009553e-07 1.65799705e-06]]
La iteración 5 es: [[1.47532560e-06 1.23845596e-06 1.23845596e-06 ... 8.11407363e-07
  8.10253524e-07 1.54186981e-06]]
La iteración 6 es: [[1.45136985e-06 1.22275244e-06 1.22339024e-06 ... 8.10468593e-07
  8.08615332e-07 1.51336681e-06]]
La iteración 7 es: [[1.44204347e-06 1.21201106e-06 1.21200531e-06 ... 8.09079590e-07
  8.07112919e-07 1.49098406e-06]]
La iteración 8 es: [[1.43581453e-06 1.20762695e-

array([[1.43134612e-06],
       [1.20337476e-06],
       [1.20352358e-06],
       ...,
       [8.08074069e-07],
       [8.05891363e-07],
       [1.47462891e-06]])

In [14]:
pr = nx.pagerank(g, alpha=0.80,max_iter=10, tol=1e-06)
pr

{1: 7.107977478931159e-07,
 6548: 3.04910068829472e-06,
 15409: 3.04910068829472e-06,
 57031: 4.8397876342509696e-06,
 13102: 4.8397876342509696e-06,
 2: 0.00014122500567711379,
 17794: 1.8659741128743804e-06,
 25202: 1.8659741128743804e-06,
 53625: 1.8659741128743804e-06,
 54582: 1.8659741128743804e-06,
 64930: 1.8659741128743804e-06,
 73764: 1.8659741128743804e-06,
 84477: 1.8659741128743804e-06,
 98628: 1.8659741128743804e-06,
 100193: 1.8659741128743804e-06,
 102355: 1.8659741128743804e-06,
 105318: 1.8659741128743804e-06,
 105730: 1.8659741128743804e-06,
 115926: 1.8659741128743804e-06,
 140864: 1.8659741128743804e-06,
 163550: 1.8659741128743804e-06,
 164599: 1.8659741128743804e-06,
 175799: 1.8659741128743804e-06,
 178642: 1.8659741128743804e-06,
 181714: 1.8659741128743804e-06,
 190453: 1.8659741128743804e-06,
 204189: 1.8659741128743804e-06,
 204604: 1.8659741128743804e-06,
 210870: 1.8659741128743804e-06,
 213966: 1.8659741128743804e-06,
 225119: 1.8659741128743804e-06,
 2415